# Create a Core Electron Configuration Input File For **dm-electron**

Create the input file for the core wave functions. This is simply an hdf5 file with the following entries:

- Number of atoms in the primitive cell
    - key: n_atoms


- Equilibrium positions (in reduced cooredinates)
    - key: eq_pos_red
    - Dim : [ n_atoms, 3 ]


- Total number of states
    - key: n_core_states
    
    
- List of Proton #'s at each site
    - key: Z_list
    - Dim : [ n_atoms ]


- Core electron configuration
    - key: core_elec_config
    - Dim : [ n_core_states, 5 ]
    - Comment: Each core state has a specified i, n, l, m, n_s (number of spins)
   
   
- Energy of the core levels
    - key: core_energy
    - Dim : [ n_core_states ]
        
Examples for both Si and Ge. Simply run the function 'generate_core_elec_config_from_MP' to generate the file automatically from the materials project database.

## Packages

In [1]:
from pymatgen.ext.matproj import MPRester
from pymatgen.core.periodic_table import Element
from pymatgen.io.cif import CifParser, CifFile
from mendeleev import element
import numpy as np
import h5py

## Functions

In [2]:
def generate_core_elec_config_from_MP(mp_api_key, mp_id, out_filename):
    """
        Creates the core electron configuration file based on a Materials Project ID #.
        
        mp_api_key is your API key for getting data from Materials Project. Get it from here:
        
        https://www.materialsproject.org/dashboard
        
    """
    
    # useful functions
    def get_shell_states(n_states, j, n, l):
    
        config = []

        n_states_left = n_states
        for m in range(-l, l + 1):
            # can you put two spins in the state?
            if n_states_left >= 2:
                n_s = 2
                config.append([j, n, l, m, 2])
                n_states_left -= 2
            # put left overs in last and leave
            elif 0 < n_states_left < 2:
                config.append([j, n, l, m, n_states_left])
                break
            else:
                break

        return config

    def generate_core_elec_config(primitive_cell_config, Z_list):

        core_elec_config = []

        for j in range(len(Z_list)):

            for key, val in element(primitive_cell_config[j]).ec.conf.items():

                shell_states = get_shell_states(val, j + 1, key[0], orbital_name_to_index[key[1]])

                core_elec_config.append(shell_states)

        return [item for sublist in core_elec_config for item in sublist]

    def get_core_energy(primitive_cell_config, core_elec_config):

        core_energy = []

        for state in core_elec_config:

            ele_name = primitive_cell_config[state[0] - 1]

            orbital_str = str(state[1])+orbital_index_to_name[state[2]]

            mp_core_energy = Element(ele_name).atomic_orbitals[orbital_str]

            core_energy.append(hartree_to_eV*mp_core_energy)

        return core_energy
    
    def save_dict_to_hdf5(dic, filename):

        with h5py.File(filename, 'w') as h5file:
            recursively_save_dict_contents_to_group(h5file, '/', dic)

    def recursively_save_dict_contents_to_group(h5file, path, dic):

        for key, item in dic.items(): 
            if isinstance(item, (np.ndarray, np.int64, np.float64, int, bytes, list)):
                h5file[path + key] = item
            elif isinstance(item, dict):
                recursively_save_dict_contents_to_group(h5file, path + key + '/', item)
            else:
                raise ValueError('Cannot save %s type'%type(item))
                
    def get_eq_pos_red(api_key, mat_id):

        with MPRester(api_key) as m:

            lattice_structure = m.get_structures(mat_id)[0].lattice.as_dict()

            cif_data = m.get_doc(mat_id)['cifs']['primitive']

            a_vecs_A = lattice_structure['matrix']
            b_vecs_A = 2*np.pi*np.linalg.inv(a_vecs_A)

            tmp_file = './tmp_file'

            f = open(tmp_file, 'w')
            f.write(cif_data)
            f.close()

            cif_f = CifParser(tmp_file)

            eq_pos_red = []

            for site in cif_f.get_structures(primitive = False)[0].as_dict()['sites']:

                eq_pos_red.append(np.array([ float(x) for x in site['abc'] ]))

            eq_pos_red = np.array(eq_pos_red)

        return eq_pos_red
    
    # constants
    hartree_to_eV = 27.211
    orbital_name_to_index = {'s': 0, 'p': 1, 'd': 2, 'f': 3}
    orbital_index_to_name = {0:'s', 1:'p', 2:'d', 3:'f'}
    
    with MPRester(my_api_key) as m:
    
        data_dict = m.get_data(mp_id)[0]

        n_atoms = data_dict['nsites']
        rho_T_g_per_cm3 = data_dict['density']
        pc_vol_A = data_dict['volume']
        mat_name = data_dict['pretty_formula']

        Z_list = []
        primitive_cell_config = []
        for j in range(len(data_dict['unit_cell_formula'].values())):

            ele_name = list(data_dict['unit_cell_formula'].keys())[j]
            n_ele = int(list(data_dict['unit_cell_formula'].values())[j])

            for i in range(n_ele):

                Z_list.append(Element(ele_name).Z)
                primitive_cell_config.append(ele_name)

        core_elec_config = generate_core_elec_config(primitive_cell_config, Z_list)
        n_core_states = len(core_elec_config)
        
        core_energy = get_core_energy(primitive_cell_config, core_elec_config)
        
        out_data_dict = {
            'n_atoms': n_atoms,
            'eq_pos_red': [],
            'n_core_states': n_core_states, 
            'Z_list': Z_list,
            'core_energy': core_energy,
            'core_elec_config': np.array(core_elec_config).T,
            'eq_pos_red': get_eq_pos_red(my_api_key, mp_id).T
        }
        
        save_dict_to_hdf5(out_data_dict, out_filename)
        
        return out_data_dict

## Si

In [3]:
# go here to get your api key : https://www.materialsproject.org/dashboard

my_api_key = "KhbT50H9pROiGqtv"

data = generate_core_elec_config_from_MP(my_api_key, 
                                         "mp-149", 
                                         'C:/Users/Tanner/Desktop/Si_core_config_test.hdf5')

print(data)

{'n_atoms': 2, 'eq_pos_red': array([[0.75, 0.5 ],
       [0.75, 0.5 ],
       [0.75, 0.5 ]]), 'n_core_states': 14, 'Z_list': [14, 14], 'core_energy': [-1773.7334158859999, -138.097348816, -95.644977918, -95.644977918, -95.644977918, -10.833760329, -4.171255823, -1773.7334158859999, -138.097348816, -95.644977918, -95.644977918, -95.644977918, -10.833760329, -4.171255823], 'core_elec_config': array([[ 1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2],
       [ 1,  2,  2,  2,  2,  3,  3,  1,  2,  2,  2,  2,  3,  3],
       [ 0,  0,  1,  1,  1,  0,  1,  0,  0,  1,  1,  1,  0,  1],
       [ 0,  0, -1,  0,  1,  0, -1,  0,  0, -1,  0,  1,  0, -1],
       [ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2]])}


## Ge

In [4]:
# go here to get your api key : https://www.materialsproject.org/dashboard

my_api_key = "KhbT50H9pROiGqtv"

data = generate_core_elec_config_from_MP(my_api_key, 
                                         "mp-32", 
                                         'C:/Users/Tanner/Desktop/Ge_core_config_test.hdf5')

print(data)

{'n_atoms': 2, 'eq_pos_red': array([[0.  , 0.25],
       [0.  , 0.25],
       [0.  , 0.25]]), 'n_core_states': 32, 'Z_list': [32, 32], 'core_energy': [-10783.528578100999, -1334.843278502, -1189.668430219, -1189.668430219, -1189.668430219, -162.217614592, -114.145301442, -114.145301442, -114.145301442, -30.403285676, -30.403285676, -30.403285676, -30.403285676, -30.403285676, -11.606117352999998, -4.078439101999999, -10783.528578100999, -1334.843278502, -1189.668430219, -1189.668430219, -1189.668430219, -162.217614592, -114.145301442, -114.145301442, -114.145301442, -30.403285676, -30.403285676, -30.403285676, -30.403285676, -30.403285676, -11.606117352999998, -4.078439101999999], 'core_elec_config': array([[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
       [ 1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,
         1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4],
